# accim: a Python library to perform building energy simulations considering adaptive comfort and data analytics

This Jupyter Notebook contains the code for the case study (Section 3 Results) of the research paper *accim: a Python library to perform building energy simulations considering adaptive comfort and data analytics*. It has been developed to ease the reproducibility of the paper, and the replicability with other IDF and EPW files or other arguments.


## 3. Results

In this case study, the aim is the analysis of energy demand resulting from the use of adaptive setpoint temperatures based on a local adaptive comfort model for office buildings in India, named IMAC-C, and the comparison with ASHRAE 55 adaptive model and static setpoints from the Indian Building Code, in different operation modes. In this study, 2 different locations, a present and a future scenario, are considered, resulting in 4 different EPW files.

The building energy model used is the 2018 IECC DOE Commercial Reference Building Prototype Small Office. It has 6 thermal zones: a thermal zone in the centre of the building (CORE_ZN), surrounded by 4 thermal zones (PERIMETER_ZN_1 to 4), all of them air-conditioned and located on the ground floor, and a non-air-conditioned attic (Figure 3). Since this is a well-known model, no further details are provided, however these are included in the folder with the IDF and EPW files, in an XLS file provided by DOE and named “PNNL_Scorecard_Prototypes_Office_Small.xls” as well as available online in https://www.energycodes.gov/prototype-building-models#Commercial.


The available EPW files in this path are:

In [1]:
from os import listdir
input_files = [i for i in listdir()]
input_epws = [i for i in input_files if i.endswith('.epw')]
print(*input_epws, sep='\n')

Current_Ahmedabad-hour.epw
Current_Shimla-hour.epw
RCP852100_Ahmedabad-hour.epw
RCP852100_Shimla-hour.epw


and the available IDF file or files are:

In [2]:
input_idfs = [i for i in input_files if i.endswith('.idf')]
print(*input_idfs, sep='\n')

SmallOffice.idf


At this point, the methodology explained in the research paper can be applied. It will be composed of the following sections and subsections:
- __Data pre-processing__
    - __EPW files preparation__
    - __ACCIS implementation__
- __Running simulations__
- __Data analysis__
    - __Tables__
    - __Data visualization__

## 3.1 Data pre-processing

At this stage, input files need to be prepared for simulation.

### 3.1.1.	EPW files preparation

First, EPWs will be formatted to a suitable name pattern for later simulation. To do so, the user can instantiate the class `rename_epw_files`. After instantiating the class, the user will need to interact with accim at the python console or CMD terminal. `confirm_deletion` argument has been set to ``False`` to avoid deletion of the original files, since these are needed to re-run the Jupyter Notebook

In [3]:
from accim.data.data_preprocessing import rename_epw_files
rename_epw_files(
    rename_dict={
        'Ahmedabad': 'Ahmedabad',
        'Shimla': 'Shimla'
    },
    confirm_deletion=False,
    confirm_renaming=True
)

Since no match has been found between RCP or SSP scenario Year and EPW file name, Present year has been assigned to the following EPW files:
Current_Ahmedabad-hour.epw
Current_Shimla-hour.epw
The geolocation process has taken: 1.74 seconds (0.43 s/EPW)

The previous and new names of the EPW files and their unique IDs are:
ID: 0 / Current_Ahmedabad-hour / India_Ahmedabad_Present
ID: 1 / Current_Shimla-hour / India_Shimla_Present
ID: 2 / RCP852100_Ahmedabad-hour / India_Ahmedabad_RCP85-2100
ID: 3 / RCP852100_Shimla-hour / India_Shimla_RCP85-2100



If any of the city or subcountry names needs some amendment (if you are not happy with any of the available options, you can exclude it from renaming at the next stage), please enter the EPW IDs separated by space; otherwise, hit enter to omit: 



The final list of previous and new names of the EPW files and their unique IDs is:
ID: 0 / Current_Ahmedabad-hour / India_Ahmedabad_Present
ID: 1 / Current_Shimla-hour / India_Shimla_Present
ID: 2 / RCP852100_Ahmedabad-hour / India_Ahmedabad_RCP85-2100
ID: 3 / RCP852100_Shimla-hour / India_Shimla_RCP85-2100



If you want to exclude some EPWs from renaming, please enter the IDs separated by space, otherwise, hit enter to continue: 


The file Current_Ahmedabad-hour has been renamed to India_Ahmedabad_Present
The file Current_Shimla-hour has been renamed to India_Shimla_Present
The file RCP852100_Ahmedabad-hour has been renamed to India_Ahmedabad_RCP85-2100
The file RCP852100_Shimla-hour has been renamed to India_Shimla_RCP85-2100


The tool renamed correctly all EPW files at the first instance and no amendments were required. The new EPWs are:

In [4]:
output_epws = [i for i in listdir() if i.endswith('.epw') if i not in input_epws]
print(*output_epws, sep='\n')

India_Ahmedabad_Present.epw
India_Ahmedabad_RCP85-2100.epw
India_Shimla_Present.epw
India_Shimla_RCP85-2100.epw


Now, the old EPWs need to be moved to a different folder to save them as a backup.

In [5]:
import shutil
for i in input_epws:
    shutil.move(i, f'backup/{i}')

### 3.1.2.	ACCIS implementation

First, let's apply 24/7 (always on) schedules to occupancy, so that we have the HVAC system on all the hours of the year.

In [6]:
from accim.utils import set_occupancy_to_always_path
set_occupancy_to_always_path(idfpath='SmallOffice.idf')

On 24/7 Schedule:Compact object was already in the model.
People Block1:PerimeterXZNX4 Number of People Schedule Name has been set to always occupied.
People Block1:CoreXZN Number of People Schedule Name has been set to always occupied.
People Block1:PerimeterXZNX2 Number of People Schedule Name has been set to always occupied.
People Block1:PerimeterXZNX1 Number of People Schedule Name has been set to always occupied.
People Block1:PerimeterXZNX3 Number of People Schedule Name has been set to always occupied.


The next step is applying the adaptive setpoint temperatures. To do so, the function ``addAccis`` needs to be used. Again, the recommended process, suitable for users with no programming background, is:
- open a CMD terminal pointing at the path where the EPW files to be renamed are located
- execute Python by entering ``py`` or ``python``
- import the package by entering ``from accim.sim import accis``
- call the function by entering ``accis.addAccis()``
- enter the required information on CMD terminal (related to the arguments that have not been specified)

In this case study, all arguments are specified as shown below:
-	``ScriptType`` is ``'vrf_mm'``, since this option adds a generic VRF system for each zone, and the input IDF was a generic model exported from DesignBuilder without any HVAC system at all.
-	``'temperature difference'`` in argument ``SupplyAirTempInputMethod`` specifies the type of supply air temperature input method for the VRF systems. This is due to the fact that, running simulations in hot weather with high cooling setpoint temperatures might result in little cooling needs, sometimes too little to be used with a generic cooling supply air temperature of 14°C.
-	``Output_keep_existing`` is set to ``False`` in order to not to generate unnecessary results.
-	``Output_type`` is set to ``'standard'``, in order to provide typical results. Other options might be used when a large amount of simulations is carried out and file size needs to be reduced, or when very specific results need to be exported for the testing of the tool.
-	``Output_freqs`` is set only to ``'hourly'``, since aggregation into other frequencies will be performed later in data analysis. If additional frequencies were required, these could be added to the list (i.e. ``['hourly', 'daily', 'monthly', 'runperiod']``)
- ``Output_gen_dataframe`` is set to False, since the generation of the panmdas DataFrame containing Output:Variable objects for output customisation is not needed.
-	``EnergyPlus_version`` is set to ``'23.1'``, since that is the version of TestModel.idf.
-	``TempCtrl`` is set to ``'temperature'``, so that the operation of the HVAC system is based on the operative temperature. If ``'pmv'`` was used, setpoints would be ±0.5.
-	``ComfStand`` requested values are ``2`` and ``7``, since these are the numbers respectively for ASHRAE 55 and IMAC-C in NV mode.
-	``CAT`` is set only to ``80``, so that the offset of setpoint temperatures from neutral temperature covers an 80% acceptability.
-	``ComfMod`` requested values are ``0`` and ``3``, since these are respectively for static and adaptive setpoints, and in this study, adaptive setpoints from IMAC-C and static setpoints from Indian Building Code are to be compared.
- ``SetpointAcc`` is set to 100, since HVAC units thermostats are considered to be accurate up to the second decimal.
- ``CoolSeasonStart`` and ``CoolSeasonEnd`` are set to dates 01/05 and 01/10 respectively, however these are not necessary in this case.
-	``HVACmode`` requested values are ``0``, ``1`` and ``2``, since those are respectively for AC, NV and MM operation modes. In this case study, may be interesting to evaluate the energy performance of MM against AC. Also, may be interesting to see the difference in comfort hours between NV and MM mode.
-	``VentCtrl``, ``VSToffset``, ``MinOToffset`` and ``MaxWindSpeed`` are arguments to control the operation of MM. In this case, the windows are opened when operative temperature exceeds the neutral temperature (``VentCtrl`` is 0) with no offset (``VSToffset`` is 0), and no restrictions in terms of minimum outdoor temperature and maximum wind speed are set (``MinOToffset`` and ``MaxWindSpeed`` are 50).
-	``ASTtol_steps``, ``ASTtol_start`` and ``ASTtol_end_input`` are mainly used to apply some tolerances to make sure there are no hours that fall outside the comfort zone by a little extent. These can also be used to generate an array of setpoint temperatures. In this case, it applies a ±0.1°C offset to heating and cooling setpoints.

In [7]:
from accim.sim import accis
accis.addAccis(
    ScriptType='vrf_mm',
    SupplyAirTempInputMethod='temperature difference',
    Output_keep_existing=False,
    Output_type='standard',
    Output_freqs=['hourly'],
    Output_gen_dataframe=False,
    EnergyPlus_version='23.1',
    TempCtrl='temperature',
    ComfStand=[2, 7],
    CAT=[80],
    ComfMod=[0, 3],
    SetpointAcc=100,
    CoolSeasonStart='01/05',
    CoolSeasonEnd='01/10',
    HVACmode=[0, 1, 2],
    VentCtrl=[0],
    VSToffset=[0],
    MinOToffset=[50],
    MaxWindSpeed=[50],
    ASTtol_steps=0.1,
    ASTtol_start=0.1,
    ASTtol_end_input=0.1,
    confirmGen=True,
)


--------------------------------------------------------
Adaptive-Comfort-Control-Implemented Model (ACCIM) v0.7.3
--------------------------------------------------------

This tool allows to apply adaptive setpoint temperatures. 
For further information, please read the documentation: 
https://accim.readthedocs.io/en/master/
For a visual understanding of the tool, please visit the following jupyter notebooks:
-    Using addAccis() to apply adaptive setpoint temperatures
https://accim.readthedocs.io/en/master/jupyter_notebooks/addAccis/using_addAccis.html
-    Using rename_epw_files() to rename the EPWs for proper data analysis after simulation
https://accim.readthedocs.io/en/master/jupyter_notebooks/rename_epw_files/using_rename_epw_files.html
-    Using runEp() to directly run simulations with EnergyPlus
https://accim.readthedocs.io/en/master/jupyter_notebooks/runEp/using_runEp.html
-    Using the class Table() for data analysis
https://accim.readthedocs.io/en/master/jupyter_notebo

As a result, 12 output IDFs are generated:

In [8]:
output_idfs = [i for i in listdir() if i.endswith('.idf') and i not in input_idfs]
print(*output_idfs, sep='\n')

SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT

However, not all of them are necessary. The IDFs that are needed are listed below (also in Table 7 of the research paper):

In [9]:
necessary_idfs = [
    'SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf',
    'SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf',
    'SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf',
    'SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf',
    'SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf'    
]
print(*necessary_idfs, sep='\n')

SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf


Thus, the remaining, shown below, should be removed to avoid unnecessary computational effort.

In [10]:
idfs_to_be_removed = [i for i in listdir() if i.endswith('.idf') and i not in necessary_idfs and i not in input_idfs]
print(*idfs_to_be_removed, sep='\n')
from os import remove
for i in idfs_to_be_removed:
    remove(i)

SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf


At this point, all EPWs have been formatted to a suitable file name and adaptive setpoint temperatures have been implemented in the input IDFs, so simulations can be run.

## 3.2.	Running simulations

Once EPWs have been formatted and adaptive setpoint temperatures have been implemented in the output IDFs, simulations can be automatically run using the function runEp. Again, the recommended process, suitable for users with no programming background, is:
- open a CMD terminal pointing at the path where the renamed EPWs and IDFs are located
- execute Python by entering ``py`` or ``python``
- import the package by entering ``from accim.run import run``
- call the function by entering ``run.runEp()``
- enter the required information on CMD terminal (related to the arguments that have not been specified)

In this case study, all arguments are specified:
- ``runOnlyAccim`` is set to ``True``, since only output IDFs need to be simulated;
- ``confirmRun`` is set to ``True``, since the number of simulation runs is known (5 IDFs x 4 EPWs = 20 runs), otherwise, if the user were not sure of the number, it could lead to some memory errors or undesired results;
- ``num_CPUs`` is set to ``4``, to be suitable for most computers;
- ``EnergyPlus_version`` is set to ``'23.1'``, since the EnergyPlus version of the output IDFs is known.

Therefore, no interaction with the user is needed and simulation can be automatically run:

In [ ]:
from accim.run import run
run.runEp(
    runOnlyAccim=True, #only runs output IDFs, that is, IDFs with "[" in its name
    confirmRun=True, #to skip confirmation
    num_CPUs=4, #to specify the number of CPUs to be used
    EnergyPlus_version='23.1', #to specify the EnergyPlus version of the IDF, and the version of EnergyPlus you are going to run
)

IDD was already set.
The IDFs we are going to run are:
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_IND IMAC C NV[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
SmallOffice[CS_INT ASHRAE55[CA_80[CM_3[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X.idf
 and the No. of IDFs is going to be 5
The sample_EPWs we are going to run are:
India_Ahmedabad_Present.epw
India_Ahmedabad_RCP85-2100.epw
India_Shimla_Present.epw
India_Shimla_RCP85-2100.epw
 and the No. of sample_EPWs is going to be 4
Therefore, the simulations are going to be:
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X[India_Ahmedabad_Present
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[HM_0[VC_X[VO_X[MT_X[MW_X[AT_0.1[NS_X[India_Ahmedabad_RCP85-2100
SmallOffice[CS_IND IMAC C NV[CA_80[CM_0[

The simulation output files have been saved in the same path. In this case study 20 simulations have been run, therefore there must be 20 CSV files ready to be analysed, which are shown below:

In [ ]:
from os import listdir
csvs = [i for i in listdir() if i.endswith('.csv') and 'Zsz.csv' not in i and 'Table.csv' not in i and 'Meter.csv' not in i]
print(*csvs, sep='\n')

## 3.3.	Data analysis

In the following subsections, the CSV files resulting from the simulation runs will be analysed, to provide a table to assess the energy demand differences, and to visualize the data in multiple figures. In this case, it is recommended to use an IDE instead of a CMD terminal.

### 3.3.1 Tables

The aim of this subsection is to create a table to see how much energy is demanded in the different settings. To do so, the first step would be to generate a DataFrame instance using the class ``Table``, which is going to be stored in variable ``dataset_runperiod``. The arguments are specified as follows:

-	``datasets`` is set to a list of all CSV files except those in which operation mode is NV (HM_1), since energy demand is 0
-	``source_frequency`` is set to ``'hourly'``, since that is the frequency that has been used when the output IDFs were generated.
-	``frequency`` is set to ``'runperiod'``, since the target table is intended to have only 1 row per simulation run
-	``frequency_agg_func`` is set to ``'sum'``, since all hourly energy demand values must be summed on a runperiod frequency basis.
-	``standard_outputs`` is set to ``True``, to use only a controlled range of outputs.
-	``level`` is set only to ``'building'``, since the target table should show the sum of energy demand values from all thermal zones.
-	``level_agg_func`` is set only to ``'sum'``, since only summed values of all zones are necessary.
-	``level_excluded_zones`` is set to ``'ATTIC:ATTIC'``. If the user opens the IDF file, 2 blocks can be found: ATTIC and Block1. In this case, the block ATTIC contains only one zone named ATTIC as well, which is not air-conditioned. Therefore, it should be omitted in the calculations.
-	``split_epw_names`` is set to ``True``, since the EPWs have previously formatted to analyse the locations and climate scenarios


In [ ]:
from accim.data.postprocessing.main import Table
dataset_runperiod = Table(
    datasets=[i for i in csvs if 'HM_1' not in i],
    source_frequency='hourly',
    frequency='runperiod',
    frequency_agg_func='sum',
    standard_outputs=True,
    idf_path='SmallOffice.idf',
    level=['building'],
    level_agg_func=['sum'],
    level_excluded_zones=['Attic:Attic'],
    split_epw_names=True,
)

If the user needs to see the DataFrame instance that has been created, it can be accessed at the variable ``df`` stored within the class ``dataset_runperiod`` as shown below:

In [ ]:
dataset_runperiod.df

Then, the method ``format_table`` needs to be executed, to filter only energy demand columns at building level, where:
-	``type_of_table`` is set to ``'custom'``, so that specifically the columns specified in custom_cols are filtered
-	``custom_cols`` is a list of the columns that should be kept, namely heating, cooling and total energy demand at building level


To know exactly the name of the energy demand columns at building level, the user can access to the DataFrame instance and search in the columns. For instance, the code below searches the string "energy demand" in all columns and print the results.

In [ ]:
energy_demand_cols = [i for i in dataset_runperiod.df.columns if 'energy demand' in i.lower()]
print(*energy_demand_cols, sep='\n')

In this case, only columns at building level are necessary, therefore these are specified when the format_table method is called below:

In [ ]:
dataset_runperiod.format_table(
    type_of_table='custom',
    custom_cols=[
        'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
        'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
        'Building_Total_Total Energy Demand (kWh/m2) (summed)'
    ]
)

Again, if the user needs to see how the DataFrame instance has been modified, it can be accessed at the variable ``df`` stored within the class ``dataset_runperiod`` as shown below:

In [ ]:
dataset_runperiod.df

At this point, the user might have a vague idea of what table can be done with the available data, especially if there is a large number of categorical variables. In order to get a clearer idea of this, the user can call the method named ``gather_vars_query``. It only takes one argument, named ``vars_to_gather``, which should be a list of the variables to be combined.

In this case, when IDFs with adaptive setpoint were generated, the arguments where more than one than one value were requested were ComfStand, ComfMod and HVACmode (``ComfStand=[2, 7], ComfMod=[0, 3], HVACmode=[0, 1, 2]``). Therefore, these are all the categorical variables that change regarding the IDFs and the possibilities that might be interesting to study, and therefore the variables that have been entered in ``vars_to_gather``.

Then, after calling ``gather_vars_query``, accim prints on screen the categorical variables that contains more than one different value (i.e. ComfMod, since values are “CM_0” and “CM_3”), and the different combinations based on the combined variables, joined by character “[“:

In [ ]:
dataset_runperiod.gather_vars_query(
    vars_to_gather=[
        'ComfStand',
        'ComfMod',
        'HVACmode'
    ]
)

At this point, the user should be aware of the possibilities to be studied, and therefore can confidently run the method ``wrangled_table`` to get the target table, where the following arguments are specified:
-	``reshaping`` is set to ``'unstack'``, since some variables in the rows should be moved to the columns.
-	``vars_to_gather`` is set to “ComfStand”, “ComfMod” and “HVACmode”, as previously showed in method ``gather_vars_query``.
-	``baseline`` is set to ``'CS_IND IMAC C NV[CM_3[HM_2'``, which is the setting that this case study intends to analyse.
-	``vars_to_keep`` is set to ``'EPW_City_or_subcountry'``, ``'EPW_Scenario'`` and ``'EPW_Year'``, since these are the remaining categorical variables apart from those specified in vars_to_gather.
-	``comparison_mode`` is only set to ``'baseline compared to others'``, since only the variance of the baseline respect to all other combinations must be shown.
-	``comparison_cols`` is only set to ``'relative'``, since absolute differences are not needed.
-	``rename_dict`` is a dictionary following the acronyms from Table 7 in research paper and available options previously shown in method ``gather_vars_query``.
-	``transpose`` is set to ``True``, since the untransposed was not in a suitable shape.


In [ ]:
dataset_runperiod.wrangled_table(
    reshaping='unstack',
    vars_to_gather=['ComfStand', 'ComfMod', 'HVACmode'],
    baseline='CS_IND IMAC C NV[CM_3[HM_2',
    vars_to_keep=['EPW_City_or_subcountry', 'EPW_Scenario', 'EPW_Year'],
    comparison_mode=['baseline compared to others'],
    comparison_cols=['relative'],
    rename_dict={
        'CS_IND IMAC C NV[CM_0[HM_0': 'Ind_Stat_AC',
        'CS_IND IMAC C NV[CM_3[HM_0': 'Ind_Adap_AC',
        'CS_IND IMAC C NV[CM_3[HM_2': 'Ind_Adap_MM',
        'CS_INT ASHRAE55[CM_3[HM_0': 'ASH_Adap_AC'
    },
    transpose=True,
    excel_filename='testing_accim'
)

Finally, an Excel spreadsheet named ``'testing_accim.xlsx'`` as specified in ``excel_filename`` argument, is created and saved in the same path where the method was called. The Excel spreadsheet is read and shown below (also in Table 8, in the research paper):

In [ ]:
import pandas as pd
output_xlsx = pd.read_excel('testing_accim.xlsx', index_col=[0, 1], header=[0, 1, 2])
output_xlsx

The user can also access the ``wrangled_table`` output at the variable stored within the class ``dataset_runperiod``, depending on the ``reshaping`` method used:
- for ``'pivot'``, ``'wrangled_df_pivoted'``
- for ``'unstack'``, ``'wrangled_df_unstacked'``,
- for ``'stack'``, ``'wrangled_df_stacked'``,
- for ``'multiindex'``, ``'wrangled_df_multiindex'``

In this case, with ``'unstack'``:

In [ ]:
dataset_runperiod.wrangled_df_unstacked

### 3.3.2.	Data visualization

The aim of this subsection is to plot figures that represent the results of the building energy simulations with adaptive setpoint temperatures. Methods ``scatter_plot``, ``scatter_plot_with_baseline`` and ``time_plot`` need to be called after method ``format_table``. However, the class Table instance created at the previous section had runperiod ``frequency``, which would be translated into very simple and not interesting figures, and besides NV mode was not considered. Therefore, a new instance will be created in this case using hourly ``frequency``, in order to plot hourly values of indoor temperature and energy demand among others.

In [ ]:
from accim.data.postprocessing.main import Table
dataset_hourly = Table(
    source_frequency='hourly',
    frequency='hourly',
    frequency_agg_func='sum',
    standard_outputs=True,
    idf_path='SmallOffice.idf',
    level=['building'],
    level_agg_func=['mean', 'sum'],
    level_excluded_zones=['Attic:Attic'],
    split_epw_names=True,
)

In this case, ``format_table`` will be called using the parameter ``type_of_table`` “temperature”, which will provide the mean operative temperature of all air conditioned zones, the adaptive setpoint temperatures, a value of running mean outdoor temperature of a single zone (since it is the same for all of them), and the building total heating and cooling energy demands.

In [ ]:
dataset_hourly.format_table(
    type_of_table='custom',
    custom_cols=[
        'Block1:PerimeterXZNX4_ASHRAE 55 Running mean outdoor temperature (°C)',
        'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
        'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
        'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
        'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
        'Building_Total_Zone Operative Temperature (°C) (mean)'
    ]
)

Now, the user can see which are the columns of the DataFrame instance:

In [ ]:
dataset_hourly.df.columns

And the DataFrame itself:

In [ ]:
dataset_hourly.df

Once ``format_table`` has been called, the methods to create the figures can follow. First, ``scatter_plot`` method will be used, in which the typical plot to show the linear regression of adaptive comfort models will be generated. As specified in the arguments below, the variables “ComfStand”, “ComfMod” and “HVACmode” were gathered and the different combinations are plotted in rows, while “EPW_City_or_subcountry” and “EPW_Scenario-Year” combinations are plotted in the columns. Data on main y-axis (left) was heating and cooling energy demand, while setpoint temperatures and operative temperature were plotted on the secondary y-axis (right), following the colours specified in the arguments. Finally, these were renamed following the dictionaries entered.

In [ ]:
dataset_hourly.scatter_plot(
    vars_to_gather_rows=['ComfStand', 'ComfMod', 'HVACmode'],
    vars_to_gather_cols=['EPW_City_or_subcountry', 'EPW_Scenario-Year'],
    data_on_x_axis='Block1:PerimeterXZNX4_ASHRAE 55 Running mean outdoor temperature (°C)',
    data_on_y_main_axis=[
        [
            'Energy Demand (kWh/m2)',
            [
                'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
                'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
            ]
        ],
    ],
    data_on_y_sec_axis=[
        [
            'Temperature (°C)',
            [
                'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
                'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
                'Building_Total_Zone Operative Temperature (°C) (mean)'
            ]
        ],
    ],
    colorlist_y_main_axis=[
        [
            'Energy Demand (kWh/m2)',
            [
                'cyan',
                'orange',
            ]
        ],
    ],
    colorlist_y_sec_axis=[
        [
            'Temperature (°C)',
            [
                'blue',
                'red',
                'green'
            ]
        ],
    ],
    best_fit_deg_y_main_axis=[
        [
            'Energy Demand (kWh/m2)',
            [
                1,
                1,
            ]
        ],
    ],
    cols_renaming_dict={
        'Ahmedabad[Present': 'Ahmedabad Present',
        'Ahmedabad[RCP85-2100': 'Ahmedabad RCP85-2100',
        'Shimla[Present': 'Shimla Present',
        'Shimla[RCP85-2100': 'Shimla RCP85-2100'
    },
    rows_renaming_dict={
        'CS_IND IMAC C NV[CM_0[HM_0': 'IND_Stat_AC',
        'CS_IND IMAC C NV[CM_3[HM_0': 'IND_Adap_AC',
        'CS_IND IMAC C NV[CM_3[HM_1': 'IND_Adap_NV',
        'CS_IND IMAC C NV[CM_3[HM_2': 'IND_Adap_MM',
        'CS_INT ASHRAE55[CM_3[HM_0': 'ASH_Adap_AC'
    },
    sharex=False,
    sharey=False,
    supxlabel='Prevailing mean outdoor temperature (°C)',
    figname='testing_scatterplot_case_study',
    figsize=4,
    ratio_height_to_width=0.5,
    dpi=300,
    confirm_graph=True
)

Using a similar code snippet but changing method to ``time_plot`` and omitting the argument ``data_on_x_axis``, a figure with time on the x-axis can be generated. In this case, the argument ``sharex`` is also omitted, since the simulation period is the same for all cases.

In [ ]:
dataset_hourly.time_plot(
    vars_to_gather_rows=['ComfStand', 'ComfMod', 'HVACmode'],
    vars_to_gather_cols=['EPW_City_or_subcountry', 'EPW_Scenario-Year'],
    data_on_y_main_axis=[
        [
            'Energy Demand (kWh/m2)',
            [
                'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
                'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
            ]
        ],
    ],
    data_on_y_sec_axis=[
        [
            'Temperature (°C)',
            [
                'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
                'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
                'Building_Total_Zone Operative Temperature (°C) (mean)'
            ]
        ],
    ],
    colorlist_y_main_axis=[
        [
            'Energy Demand (kWh/m2)',
            [
                'cyan',
                'orange',
            ]
        ],
    ],
    colorlist_y_sec_axis=[
        [
            'Temperature (°C)',
            [
                'blue',
                'red',
                'green'
            ]
        ],
    ],
    cols_renaming_dict={
        'Ahmedabad[Present': 'Ahmedabad Present',
        'Ahmedabad[RCP85-2100': 'Ahmedabad RCP85-2100',
        'Shimla[Present': 'Shimla Present',
        'Shimla[RCP85-2100': 'Shimla RCP85-2100'
    },
    rows_renaming_dict={
        'CS_IND IMAC C NV[CM_0[HM_0': 'IND_Stat_AC',
        'CS_IND IMAC C NV[CM_3[HM_0': 'IND_Adap_AC',
        'CS_IND IMAC C NV[CM_3[HM_1': 'IND_Adap_NV',
        'CS_IND IMAC C NV[CM_3[HM_2': 'IND_Adap_MM',
        'CS_INT ASHRAE55[CM_3[HM_0': 'ASH_Adap_AC'
    },
    sharey=True,
    figname='testing_timeplot_case_study',
    figsize=4,
    ratio_height_to_width=0.5,
    dpi=300,
    confirm_graph=True
)

Finally, using ``scatter_plot_with_baseline`` a different scatter plot can be generated. In this case, it is mainly used to compare some combination of variables gathered in columns with the remaining variants. In the example below, the total heating and cooling energy demands will be compared using the combination ``'CS_IND IMAC C NV[CM_3[HM_2'`` (or IND_Adap_MM) as the baseline, since it is expected to provide the greater energy savings. The same ``Table`` instance will be used, however, to avoid plotting an empty figure, since energy demand is null in the combination ``'CS_IND IMAC C NV[CM_3[HM_1'`` (or IND_Adap_NV), since it is on naturally ventilated mode, the argument ``detailed_cols`` is used. In this argument, all the combinations to be plotted are specified, that is, all of them except the NV and the baseline.

In [ ]:
dataset_hourly.scatter_plot_with_baseline(
    vars_to_gather_rows=['EPW_City_or_subcountry', 'EPW_Scenario-Year'],
    vars_to_gather_cols=['ComfStand', 'ComfMod', 'HVACmode'],
    detailed_cols=[
        'CS_IND IMAC C NV[CM_0[HM_0',
        'CS_IND IMAC C NV[CM_3[HM_0',
        'CS_INT ASHRAE55[CM_3[HM_0'
    ],
    data_on_y_axis_baseline_plot=[
                'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
                'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
    ],
    colorlist_baseline_plot_data=[
                'blue',
                'red'
    ],
    baseline='CS_IND IMAC C NV[CM_3[HM_2',
    rows_renaming_dict={
        'Ahmedabad[Present': 'Ahmedabad Present',
        'Ahmedabad[RCP85-2100': 'Ahmedabad RCP85-2100',
        'Shimla[Present': 'Shimla Present',
        'Shimla[RCP85-2100': 'Shimla RCP85-2100'
    },
    cols_renaming_dict={
        'CS_IND IMAC C NV[CM_0[HM_0': 'IND_Stat_AC',
        'CS_IND IMAC C NV[CM_3[HM_0': 'IND_Adap_AC',
        'CS_INT ASHRAE55[CM_3[HM_0': 'ASH_Adap_AC'
    },
    supxlabel='IND_Adap_MM Hourly Energy Demand (kWh/m²)',
    supylabel='Reference Hourly Energy Demand (kWh/m²)',
    figname='testing_scatterplotbaseline_case_study',
    figsize=3,
    dpi=300,
    confirm_graph=True
)

Finally, so that this jupyter notebook can be properly run again, all files different than the input ones will be deleted.

In [ ]:
for i in input_epws:
    shutil.move(f'backup/{i}', i)

In [ ]:
files_to_delete = [i for i in listdir() if i not in input_files]
print(*files_to_delete, sep='\n')

In [ ]:
for i in files_to_delete:
    remove(i)